In [14]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors

# Assuming df is your original DataFrame read from JSON
df = pd.read_json("ducks.json")

# Select relevant columns
df_selected = df[['productID', 'duckDetails', 'additionalFeatures', 'productName']].copy()

# Extract specific values from nested dictionaries
df_selected['price'] = df_selected['duckDetails'].apply(lambda x: x['price'])
df_selected['style'] = df_selected['duckDetails'].apply(lambda x: x['style'])
df_selected['speed'] = df_selected['duckDetails'].apply(lambda x: x['speed'])
df_selected['size'] = df_selected['duckDetails'].apply(lambda x: x['size'])
df_selected['condition'] = df_selected['duckDetails'].apply(lambda x: x['condition'])
df_selected['buoyancy'] = df_selected['additionalFeatures'].apply(lambda x: x['buoyancy'])

# Drop columns that are no longer needed
df_selected.drop(['duckDetails', 'additionalFeatures', 'productName'], axis=1, inplace=True)

# Apply one-hot encoding to categorical variables
df_encoded = pd.get_dummies(df_selected, columns=['speed', 'style', 'size', 'condition', 'buoyancy'])

knn = NearestNeighbors(n_neighbors=3).fit(df_encoded)

In [17]:
selected_product_features = df_encoded[df_encoded['productID'] == 1]
selected_product_index = selected_product_features.index[0]
distances, indices = knn.kneighbors(selected_product_features)
recommended_product_indices = indices.flatten()
recommended_products = df_selected.iloc[recommended_product_indices]

print("Selected Product:")
print(df_selected.iloc[selected_product_index])
print("\nRecommended Products:")
print(recommended_products)

Selected Product:
productID        1
price            5
style         food
speed         slow
size         large
condition      new
buoyancy     False
Name: 1, dtype: object

Recommended Products:
   productID  price   style    speed   size condition  buoyancy
1          1      5    food     slow  large       new     False
2          2      3  pirate  average  large       new     False
3          3      3  sports  average  large      used     False
